# Sprint 2:

## Espacio de codificación de soluciones


In [ ]:
!pip install spicy
!pip install scikit-posthocs
!pip install optuna
!pip install openpyxl

In [ ]:
import numpy as np
import networkx as nx
from scipy import sparse

import copy
import time

import sqlite3
import pandas as pd
import itertools

import optuna

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import scikit_posthocs as sp

import openpyxl

In [ ]:
connect = sqlite3.connect(r'C:\Users\gaizk\Desktop\Entrega HB\BBDD\database_50_sample.sqlite')
query = """
SELECT pa.paper_id, pa.author_id, a.name
FROM paper_authors AS pa JOIN papers AS p ON pa.paper_id = p.id
JOIN authors as a ON pa.author_id = a.id
WHERE p.Year BETWEEN '2014' AND '2015'
"""
df = pd.read_sql(query, connect)

G2 = nx.Graph()

# Transform
for p, a in df.groupby('paper_id')['name']:
    for u, v in itertools.combinations(a, 2):
        if G2.has_edge(u, v):
            G2[u][v]['weight'] +=1
        else:
            G2.add_edge(u, v, weight=1)

# **Funcion de modularidad vectorizada**

In [ ]:
class GraphProblem:
    def __init__(self, G_initial):
        """
        Prepara los datos del grafo UNA sola vez para no repetir cálculos.
        """
        self.G = G_initial
        self.nodes = list(self.G.nodes())
        self.num_nodes = len(self.nodes)
        self.edges = list(self.G.edges())

        self.dictionary = self.hashMap(self.G)

        # 1. Matriz de Adyacencia (A)
        self.A = nx.adjacency_matrix(self.G)

        # 2. Vector de Grados (degrees)
        self.degrees = np.array([val for (node, val) in self.G.degree()])

        # 3. Constantes
        self.m = self.G.number_of_edges()
        self.two_m = 2 * self.m

    def get_modularity(self, solution, k):
        """
        Calcula la modularidad de una solución dada.
        """
        sum = 0
        for i in range(k):
          Eii, Ai = self.funcion_Eii_Ai(self.G, solution, self.dictionary, i, self.m)
          sum += (Eii - Ai**2)
        return sum

    def delta_optima(self, comunidad, v, i):
      """
      Devuelve 1 si el nodo v pertenece a la comunidad i, 0 en caso contrario.
      """
        if comunidad[v] == i: return 1
        else: return 0

    def funcion_Eii_Ai(self, G, comunidad, myDict, i, m, weight='weight'):
        """
        Calcula Eii y Ai para una comunidad i.
        """
        Eii = 0
        Ai = 0
        two_m = 2 * m
        for v in G.nodes(): #Los nodos
            for w in G.neighbors(v): # Los vecinos de v
                Eii += self.delta_optima(comunidad, myDict.get(v), i) * self.delta_optima(comunidad, myDict.get(w), i)
            Ai += G.degree(v) * self.delta_optima(comunidad, myDict.get(v), i)
        return (Eii / two_m) , (Ai / two_m)

    def hashMap(self, G):
        """
        Crea un diccionario que mapea nodos(autores) a enteros.
        """
        myDict = {}
        for index, elem in enumerate(G.nodes):
            myDict[elem] = index
        return myDict

# Clase padre

In [ ]:
class Algorithm:
    def __init__(self, grafo):
        self.grafo = grafo
        self.best_solution = None
        self.best_fitness = -np.inf
        self.history = [] # Para guardar la evolución

    def run(self):
        """
        Método vacío que cada algoritmo rellenará a su manera.
        """
        raise NotImplementedError("¡Tienes que implementar este método en la clase hija!")

    def save_solution(self, solution, fitness):
        """
        Guarda la solución si es la mejor encontrada hasta ahora.
        """
        if fitness > self.best_fitness:
            self.best_fitness = fitness
            self.best_solution = np.copy(solution)

**RANDOM SEARCH**

In [ ]:
class RandomSearch(Algorithm):
    def __init__(self, grafo, k, max_evaluations):
        """
        Algoritmo de Búsqueda Aleatoria.
        Recibe el grafo, el número de comunidades (k) y el máximo de evaluaciones permitidas.
        """
        super().__init__(grafo)
        self.k = k
        self.max_evaluations = max_evaluations

    def run(self):
        evaluations = 0
        while evaluations < self.max_evaluations:
            # Generar solución aleatoria
            solution = np.random.randint(0, self.k, size=self.grafo.num_nodes)
            # Evaluar modularidad
            fitness = self.grafo.get_modularity(solution, self.k)
            # Guardar si es la mejor solución
            self.save_solution(solution, fitness)
            evaluations += 1
            self.history.append(self.best_fitness)

**GRASP**

In [ ]:
class GRASP(Algorithm):
    def __init__(self, grafo, k=5):
        super().__init__(grafo)
        self.k = k

    def run(self):
        # 1. Solución aleatoria inicial
        current_solution = np.random.randint(0, self.k, size=self.grafo.num_nodes)

        # 2. Orden aleatorio
        nodes_random_order = np.random.permutation(self.grafo.num_nodes)

        for node in nodes_random_order:
            original_comm = current_solution[node]

            # El "mejor" actual es donde está el nodo ahora mismo
            best_comm_for_node = original_comm
            # Calculamos la base antes de probar movimientos
            current_base_q = self.grafo.get_modularity(current_solution, self.k)
            best_q_for_node = current_base_q

            # PROBAMOS OTRAS COMUNIDADES
            for comm in range(self.k):
                if comm == original_comm: continue

                #Aplicamos el cambio y evaluamos
                current_solution[node] = comm
                new_q = self.grafo.get_modularity(current_solution, self.k)

                # Si mejora, guardamos este movimiento como el candidato
                if new_q > best_q_for_node:
                    best_q_for_node = new_q
                    best_comm_for_node = comm

                # Revertimos el cambio por si no mejora para mantener la solucion original
                current_solution[node] = original_comm

            # 4. Aplicar el mejor movimiento encontrado para este nodo
            # Si no encontramos nada mejor, best_comm_for_node sigue siendo original_comm
            if best_comm_for_node != original_comm:
                current_solution[node] = best_comm_for_node

        # Guardar y retornar
        final_fitness = self.grafo.get_modularity(current_solution, self.k)
        self.save_solution(current_solution, final_fitness)

        return final_fitness, current_solution

In [ ]:
# --- TEST DEL CONSTRUCTIVO ---
G = GraphProblem(G2)
greedy = GRASP(G, k=5)

t0 = time.time()
fit, sol = greedy.run()
t1 = time.time()

print(f"Modularidad obtenida: {fit:.5f}")
print(f"Tiempo en construir 1 solución: {t1-t0:.4f} s")

**SA**

In [ ]:
class SimulatedAnnealing(Algorithm):
    def __init__(self, grafo, k = 5, initial_solution=None, max_evaluations=100000):
        super().__init__(grafo) # Pasamos el grafo al padre
        self.max_evaluations = max_evaluations
        self.k = k

        # Si no nos dan solución inicial, creamos una aleatoria
        if initial_solution is None:
            self.current_solution = np.random.randint(0, self.k, size=self.grafo.num_nodes)
        else:
            self.current_solution = np.copy(initial_solution)

        # Parámetros de la temperatura y del valor de cada step
        self.initial_temp = 1.0
        self.final_temp = 0.001

        # T_final = T_inicial * alpha**n --> alpha = (T_final / T_inicial)**(1/n), donde n = max_evals
        # alpha = step
        self.alpha = (self.final_temp / self.initial_temp) ** (1 / self.max_evaluations)

    # Cambiamos la comunidad de un vecino al azar y devolvemos el vecino
    def get_neighbor(self, current_sol):
      """
      Genera un vecino moviendo UN solo nodo de comunidad.
      """
        neighbor = np.copy(current_sol)

        node_idx = np.random.randint(0, self.grafo.num_nodes)

        current_comm = neighbor[node_idx]
        new_comm = np.random.randint(0, self.k)

        while new_comm == current_comm:
            new_comm = np.random.randint(0, self.k)

        neighbor[node_idx] = new_comm
        return neighbor

    def run(self):
        # Evaluacion inicial
        current_fitness = self.grafo.get_modularity(self.current_solution, self.k)
        self.save_solution(self.current_solution, current_fitness)

        temp = self.initial_temp
        evaluations = 0

        while evaluations < self.max_evaluations:
            # Generamos nuevo candidato
            neighbor = self.get_neighbor(self.current_solution)
            neighbor_fitness = self.grafo.get_modularity(neighbor, self.k)

            # Calculamos la diferencia de calidad
            # Si es mejor el vecino que current --> delta > 0; else --> delta < 0
            delta = neighbor_fitness - current_fitness

            # Aceptaremos solucion cuando:
              # Sea mejor que la actual
              # Aunque sea peor, esto se cumpla: np.random.random() < np.exp(delta / temp)
            accept = False
            if delta > 0: # El vecino es mejor
                accept = True
            else: # El vecino es peor
                probability = np.exp(delta / temp)
                if np.random.random() < probability:
                    accept = True

            if accept:
                self.current_solution = neighbor
                current_fitness = neighbor_fitness
                self.save_solution(neighbor, neighbor_fitness)

            # Actualizamos temperatura y evaluaciones
            temp *= self.alpha
            evaluations += 1

            #Guardamos cada 1000 iteraciones para no ocupar mucha memoria
            if evaluations % 1000 == 0:
                self.history.append(self.best_fitness)

        return self.best_fitness, self.best_solution

In [ ]:
# --- TEST DEL SIMULATED ANNEALING ---
print("Iniciando Simulated Annealing...")

sa_algorithm = SimulatedAnnealing(G, max_evaluations=10000)

t0 = time.time()
fit_sa, sol_sa = sa_algorithm.run()
t1 = time.time()

print(f"Mejor Modularidad SA: {fit_sa:.5f}")
print(f"Tiempo total (10k evals): {t1-t0:.4f} s")
print(f"Velocidad: {10000/(t1-t0):.0f} iteraciones/segundo")

**EDA**

In [ ]:
class EDA(Algorithm):
    def __init__(self, grafo, k = 5, max_evaluations=100000, pop_size=50, elite_percent=0.5):
        super().__init__(grafo)
        self.k = k
        self.max_evaluations = max_evaluations
        self.pop_size = pop_size

        # Cuántos individuos seleccionamos para aprender (Elite)
        self.elite_size = int(pop_size * elite_percent)

        # MATRIZ DE PROBABILIDAD (Modelo)
        # Inicialmente: Todos los nodos tienen igual probabilidad de ir a cualquier comunidad (1/k)
        # Shapes: (Nodos x Comunidades)
        self.prob_matrix = np.ones((self.grafo.num_nodes, self.k)) / self.k

    def sample_population(self):
        """
        Genera una población entera basada en la matriz de probabilidades actual.
        """
        population = []

        # Iteramos por nodo y elegimos comunidad para TODOS los individuos a la vez.
        # Matriz temporal: (Población x Nodos)
        pop_matrix = np.zeros((self.pop_size, self.grafo.num_nodes), dtype=int)

        for node in range(self.grafo.num_nodes):
            # Probabilidades de este nodo para las k comunidades
            probs = self.prob_matrix[node]
            probs = probs / np.sum(probs) # Para evitar errores

            # Elegimos comunidad para "node" (en todas las soluciones) en funcion de "probs" (las probabilidades de cada comunidad para ese nodo)
            choices = np.random.choice(np.arange(self.k), size=self.pop_size, p=probs)
            pop_matrix[:, node] = choices

        # Convertimos la matriz en una lista de arrays (formato que usamos en nuestros algoritmos)
        population = [individual for individual in pop_matrix]
        return population

    def update_probability_model(self, elite_pop):
        """
        Actualiza la matriz de probabilidades basándose en la frecuencia de la élite.
        """
        new_probs = np.zeros((self.grafo.num_nodes, self.k))

        elite_matrix = np.array(elite_pop) # elite_pop es una lista de listas

        for k in range(self.k):
            # Sumamos cuántas veces aparece la comunidad 'k' en cada columna (nodo)
            # (elite_matrix == k) crea una matriz de True/False (Al sumar, True = 1 y False = 0)
            # sum(axis=0) suma verticalmente (Columnas)
            count_per_node = np.sum(elite_matrix == k, axis=0)

            # Asignamos las probabilidades
            new_probs[:, k] = count_per_node

        # Sumamos 1e-10 (epsilon) para evitar divisiones por cero (Si no aparece uan comunidad)
        epsilon = 1e-10
        new_probs = new_probs + epsilon

        # Normalizamos los valores
        row_sums = new_probs.sum(axis=1, keepdims=True)
        self.prob_matrix = new_probs / row_sums

    def run(self):
        evaluations = 0
        generation = 0

        while evaluations < self.max_evaluations:
            # 1. Sampling
            population = self.sample_population()

            # 2. Evaluación
            fitness_values = []
            for ind in population:
                fit = self.grafo.get_modularity(ind, self.k)
                fitness_values.append(fit)

                # Guardamos si es récord histórico
                self.save_solution(ind, fit)

            evaluations += self.pop_size

            # 3. Selección (Elitismo)
            # Ordenamos índices de mayor a menor fitness
            sorted_indices = np.argsort(fitness_values)[::-1]

            # Nos quedamos con los mejores índices
            elite_indices = sorted_indices[:self.elite_size]
            elite_population = [population[i] for i in elite_indices]

            # 4. Aprendizaje (Learning)
            self.update_probability_model(elite_population)

            generation += 1
            if evaluations % 1000 < self.pop_size: # Guardar historial aprox cada 1000 evals
                self.history.append(self.best_fitness)

        return self.best_fitness, self.best_solution

In [ ]:
print("Iniciando EDA...")

eda_algorithm = EDA(G, k=5, max_evaluations=5000, pop_size=50)

t0 = time.time()
fit_eda, sol_eda = eda_algorithm.run()
t1 = time.time()

print(f"Mejor Modularidad EDA: {fit_eda:.5f}")
print(f"Tiempo total (5k evals): {t1-t0:.4f} s")
print(f"Velocidad: {5000/(t1-t0):.0f} evaluaciones/segundo")

# **OPTUNA**

In [ ]:
def objective_sa(trial):
    """
    Esta función es la que Optuna ejecutará muchas veces.
    Su misión: Probar UNA configuración y devolver qué tal funcionó.
    """

    # 1. OPTUNA SUGIERE PARÁMETROS
    # Le pedimos que elija una temperatura inicial (debe ser grande)
    initial_temp = trial.suggest_float("initial_temp", 1.0, 100.0)

    # Le pedimos una temperatura final (debe ser pequeña)
    # log=True significa que explora escalas logarítmicas (0.001, 0.01, 0.1...)
    final_temp = trial.suggest_float("final_temp", 1e-5, 1e-1, log=True)

    # 2. INSTANCIAS
    # Usamos pocas evaluaciones para que sea rápida.
    sa = SimulatedAnnealing(
        grafo=G,
        max_evaluations=2000
    )

    # Sobreecribimos los parámetros sugeridos manualmente porque no los pusimos como entrada del algoritmo
    sa.initial_temp = initial_temp
    sa.final_temp = final_temp
    sa.alpha = (sa.final_temp / sa.initial_temp) ** (1 / sa.max_evaluations)

    # 3. EJECUTAMOS EL ALGORITMO
    fitness, _ = sa.run()

    # 4. DEVOLVEMOS EL RESULTADO (Optuna MAXIMIZARA esto)
    return fitness

# --- LANZAMIENTO DEL ESTUDIO ---
# Creamos el estudio. direction="maximize" porque queremos maximizar.
study_sa = optuna.create_study(direction="maximize")

print("Iniciando calibración con Optuna...")
# Ejecuta la función objective_sa 50 veces probando valores diferentes
study_sa.optimize(objective_sa, n_trials=50)

print("\n--- RESULTADOS ---")
print(f"Mejores parámetros encontrados: {study_sa.best_params}")
print(f"Mejor modularidad conseguida: {study_sa.best_value}")

In [ ]:
def objective_eda(trial):
    """
    Función objetivo para calibrar el EDA.
    """

    # 1. RANGOS DE BÚSQUEDA
    # Tamaño de población: Optuna probará enteros entre 10 y 100
    pop_size = trial.suggest_int("pop_size", 10, 100)

    # Porcentaje de élite: Probamos entre el 10% (0.1) y el 50% (0.5)
    elite_percent = trial.suggest_float("elite_percent", 0.1, 0.5)

    # 2. CONFIGURACIÓN DEL ALGORITMO
    eda = EDA(
        grafo=G,
        max_evaluations=2000,
        pop_size=pop_size,
        elite_percent=elite_percent
    )

    # 3. EJECUCIÓN
    fitness, _ = eda.run()

    return fitness

# --- LANZAMIENTO DEL ESTUDIO EDA ---
study_eda = optuna.create_study(direction="maximize")

print("Calibrando EDA con Optuna (usando G)...")
study_eda.optimize(objective_eda, n_trials=50)

print("\n--- RESULTADOS EDA ---")
print(f"Mejores parámetros: {study_eda.best_params}")
print(f"Mejor modularidad: {study_eda.best_value}")

In [ ]:
# --- CONFIGURACIÓN DEL EXPERIMENTO (Valores de Optuna) ---
SA_PARAMS = {'initial_temp': 3.5293443380827867, 'final_temp': 1.0610698101521354e-05}
EDA_PARAMS = {'pop_size': 81, 'elite_percent': 0.2517414893286486}

# Configuración general del enunciado
MAX_EVALUATIONS = 10**5
N_REPETICIONES = 5
K_VALUES = np.arange(2, 101) # Valores de K a probar (Del 2 al 100)

results_list = []

print(f"Iniciando Experimentación...")
print(f"Algoritmos: GRASP, SA, EDA")
print(f"Repeticiones por configuración: {N_REPETICIONES}")

# --- BUCLE PRINCIPAL ---
for k in K_VALUES:
    print(f"\n--- Probando con K={k} comunidades ---")

    for rep in range(N_REPETICIONES):

        # 1. GRASP (Constructivo)
        greedy = GRASP(G, k=k)
        t0 = time.time()
        fit_greedy, _ = greedy.run() # No necesitamos la solución, solo el fitness
        t1 = time.time()

        results_list.append({
            "Algorithm": "GRASP",
            "k": k,
            "Repetition": rep,
            "Fitness": fit_greedy,
            "Time": t1 - t0
        })

        # 2. SIMULATED ANNEALING
        sa = SimulatedAnnealing(G, k=k, max_evaluations=MAX_EVALUATIONS)
        # Inyectamos los parámetros optimos
        sa.initial_temp = SA_PARAMS['initial_temp']
        sa.final_temp = SA_PARAMS['final_temp']
        sa.alpha = (sa.final_temp / sa.initial_temp) ** (1 / sa.max_evaluations)

        t0 = time.time()
        fit_sa, _ = sa.run()
        t1 = time.time()

        results_list.append({
            "Algorithm": "Simulated Annealing",
            "k": k,
            "Repetition": rep,
            "Fitness": fit_sa,
            "Time": t1 - t0
        })

        # 3. EDA
        eda = EDA(G, k=k, max_evaluations=MAX_EVALUATIONS,
                  pop_size=EDA_PARAMS['pop_size'],
                  elite_percent=EDA_PARAMS['elite_percent'])

        t0 = time.time()
        fit_eda, _ = eda.run()
        t1 = time.time()

        results_list.append({
            "Algorithm": "EDA",
            "k": k,
            "Repetition": rep,
            "Fitness": fit_eda,
            "Time": t1 - t0
        })

# Guardamos los resultados en un DataFrame
df_results = pd.DataFrame(results_list)

print("\n¡Experimento finalizado! 🏁")

In [ ]:
# Configuración estética para informes
sns.set_theme(style="whitegrid")
plt.rcParams.update({'figure.figsize': (12, 6), 'font.size': 12})

print("\n" + "="*50)
print("GENERANDO INFORME ESTADÍSTICO Y GRÁFICO")
print("="*50)

# --- 1. TABLAS DESCRIPTIVAS (Medias y Varianzas) ---
# Agrupamos por Algoritmo y K para ver la tendencia
summary_table = df_results.groupby(['Algorithm', 'k'])['Fitness'].agg(['mean', 'std', 'var', 'min', 'max']).reset_index()

# Guardamos en un Excel:
summary_table.to_excel("tabla_estadisticas.xlsx")

# --- 2. VISUALIZACIÓN GLOBAL (Evolución vs K) ---
# Gráfico A: Evolución del Fitness (Modularidad) según K
plt.figure(figsize=(14, 7))
sns.lineplot(data=df_results, x='k', y='Fitness', hue='Algorithm', style='Algorithm', markers=True, dashes=False)
plt.title('Impacto del número de comunidades (K) en la Modularidad (Fitness)', fontsize=16)
plt.ylabel('Modularidad (Promedio)')
plt.xlabel('Número de Comunidades (K)')
plt.legend(title='Algoritmo')
plt.show()

# Gráfico B: Evolución del Tiempo de Ejecución
plt.figure(figsize=(14, 7))
sns.lineplot(data=df_results, x='k', y='Time', hue='Algorithm')
plt.title('Coste Computacional vs K', fontsize=16)
plt.ylabel('Tiempo (segundos)')
plt.xlabel('Número de Comunidades (K)')
plt.yscale('log') # Escala logarítmica porque EDA suele tardar mucho más que Greedy
plt.show()

# --- 3. ANÁLISIS PROFUNDO EN EL MEJOR K ---
# Vamos a elegir la K donde se obtuvo la MEJOR modularidad global.
best_row = df_results.loc[df_results['Fitness'].idxmax()]
best_k = best_row['k']

print(f"\nLa K con mejor rendimiento global fue K={best_k}.")

# Filtramos datos solo para esa K
df_best_k = df_results[df_results['k'] == best_k].copy()

# Gráfico C: Boxplot (Cajas y Bigotes) para ver estabilidad en la mejor K
plt.figure(figsize=(10, 6))
sns.boxplot(data=df_best_k, x='Algorithm', y='Fitness', palette="Set3")
sns.swarmplot(data=df_best_k, x='Algorithm', y='Fitness', color=".25", size=4) # Puntos individuales
plt.title(f'Dispersión de Resultados para K={best_k}', fontsize=16)
plt.show()

# Gráfico D: KDE (Densidad) - Muestra qué tan probable es obtener un buen resultado
plt.figure(figsize=(10, 6))
sns.kdeplot(data=df_best_k, x='Fitness', hue='Algorithm', fill=True, common_norm=False, palette="Set2")
plt.title(f'Distribución de Densidad del Fitness (K={best_k})', fontsize=16)
plt.xlabel('Fitness (Modularidad)')
plt.show()
